In [1]:
import numpy as np
import pandas as pd
import seaborn as sb

In [2]:
df = pd.read_csv("mafat_simp.csv")

In [3]:
df.head()

,segment_id,track_id,geolocation_type,geolocation_id,sensor_id,snr_type,date_index,target_type
0,0,11,C,1,1,HighSNR,27,animal
1,1,16,C,1,1,HighSNR,27,animal
2,2,16,C,1,1,HighSNR,27,animal
3,3,29,C,1,1,HighSNR,27,animal
4,4,29,C,1,1,HighSNR,27,animal


In [4]:
pd.Categorical(df.track_id)

[11, 16, 16, 29, 29, ..., 281988, 281988, 281988, 281988, 281999]
Length: 106610
Categories (4228, int64): [0, 1, 2, 3, ..., 281940, 281950, 281988, 281999]

In [5]:
pd.Categorical(df.geolocation_type)

[C, C, C, C, C, ..., D, D, D, D, D]
Length: 106610
Categories (3, object): [A, C, D]

In [6]:
pd.Categorical(df.geolocation_id)

[1, 1, 1, 1, 1, ..., 7, 7, 7, 7, 7]
Length: 106610
Categories (7, int64): [1, 2, 3, 4, 5, 6, 7]

In [7]:
pd.Categorical(df.sensor_id)

[1, 1, 1, 1, 1, ..., 13, 13, 13, 13, 13]
Length: 106610
Categories (13, int64): [1, 2, 3, 4, ..., 10, 11, 12, 13]

In [8]:
pd.Categorical(df.snr_type)

[HighSNR, HighSNR, HighSNR, HighSNR, HighSNR, ..., SynthSNR, SynthSNR, SynthSNR, SynthSNR, SynthSNR]
Length: 106610
Categories (3, object): [HighSNR, LowSNR, SynthSNR]

In [9]:
pd.Categorical(df.date_index)

[27, 27, 27, 27, 27, ..., 3, 3, 3, 3, 3]
Length: 106610
Categories (32, int64): [2, 3, 4, 5, ..., 30, 31, 32, 33]

In [10]:
pd.Categorical(df.target_type)

[animal, animal, animal, animal, animal, ..., human, human, human, human, human]
Length: 106610
Categories (2, object): [animal, human]

In [11]:
df2 = pd.get_dummies(df,columns=['geolocation_type'],drop_first=True)
df2.head().T

,0,1,2,3,4
segment_id,0,1,2,3,4
track_id,11,16,16,29,29
geolocation_id,1,1,1,1,1
sensor_id,1,1,1,1,1
snr_type,HighSNR,HighSNR,HighSNR,HighSNR,HighSNR
date_index,27,27,27,27,27
target_type,animal,animal,animal,animal,animal
geolocation_type_C,1,1,1,1,1
geolocation_type_D,0,0,0,0,0


In [12]:
df2 = pd.get_dummies(df2,columns=['geolocation_id'],drop_first=True)

In [13]:
df2 = pd.get_dummies(df2,columns=['sensor_id'],drop_first=True)

In [14]:
df2 = pd.get_dummies(df2,columns=['snr_type'],drop_first=True)

In [15]:
df2 = pd.get_dummies(df2,columns=['date_index'],drop_first=True)

In [16]:
df2['target_type']=np.where(df2['target_type'] =='human', 1, df2['target_type'])
df2['target_type']=np.where(df2['target_type'] =='animal', 0, df2['target_type'])
df2['target_type']=df2['target_type'].astype('int')

In [17]:
df3 = df2.copy()
df3.drop(['segment_id'],axis=1,inplace=True)

In [18]:
import sklearn.linear_model as sklin


In [19]:
model = sklin.LogisticRegression()

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df3.drop('target_type',axis=1), df3['target_type'], test_size=0.33, random_state=47)

In [22]:
model.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
pred = model.predict(X_test)

In [24]:
np.mean(pred != y_test)

0.006139503155022455

In [25]:
import sklearn.metrics as skmet

In [26]:
skmet.confusion_matrix(y_test,pred)

array([[ 2404,    91],
       [  125, 32562]], dtype=int64)

In [27]:
print(skmet.classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      2495
           1       1.00      1.00      1.00     32687

    accuracy                           0.99     35182
   macro avg       0.97      0.98      0.98     35182
weighted avg       0.99      0.99      0.99     35182



In [28]:
model.coef_

array([[ 4.56917422e-04,  1.85504362e+00,  6.69700033e-01,
        -1.19610328e+00, -2.68579075e+00,  6.62303390e-01,
         4.40883634e-03,  3.97770860e-03,  3.41893431e-03,
        -1.19610328e+00, -2.68579075e+00,  6.62303390e-01,
         1.74506421e-03,  1.78773115e-03,  8.76040990e-04,
         1.72093118e-03,  7.35324798e-04,  1.52145262e-03,
         1.20283385e-03,  1.34166242e-03,  8.74438036e-04,
        -1.98056647e+00,  1.80571423e-01,  3.41893431e-03,
         4.40883634e-03, -5.11210557e-02, -1.61607122e-01,
        -3.23711354e-01, -2.99280772e-01, -1.59116157e-01,
        -1.35040147e-01, -2.87204802e-02, -5.53246943e-02,
        -2.35295210e-01, -4.47120664e-01, -5.01819763e-01,
        -1.74391412e-01, -2.44599189e-01, -7.32219774e-02,
        -2.85327930e-01, -3.93930949e-01, -1.58644708e-01,
        -4.25196512e-03, -1.49368474e-01, -1.16692863e+00,
         2.31489944e+00, -1.16467573e+00,  4.56284695e-01,
         1.41105501e+00,  1.60042025e+00, -2.04698099e+0

In [29]:
model.intercept_

array([-1.44379634])

In [30]:
df2['proba'] = model.predict_proba(df2[X_train.columns])[:,1]

In [31]:
df2['true']=df2['target_type']
df2['predicted']=pred = model.predict(df3.drop('target_type',axis=1))

In [32]:
df2[['segment_id','true','predicted','proba']].head(10)

,segment_id,true,predicted,proba
0,0,0,1,0.705278
1,1,0,1,0.705752
2,2,0,1,0.705752
3,3,0,1,0.706984
4,4,0,1,0.706984
5,5,0,1,0.706984
6,6,0,1,0.706984
7,7,0,1,0.706984
8,8,0,1,0.706984
9,9,0,1,0.706984


In [33]:
df2[['segment_id','true','predicted','proba']].tail(10)

,segment_id,true,predicted,proba
106600,2064765,1,1,1.0
106601,2064766,1,1,1.0
106602,2064767,1,1,1.0
106603,2064768,1,1,1.0
106604,2064769,1,1,1.0
106605,2064770,1,1,1.0
106606,2064771,1,1,1.0
106607,2064772,1,1,1.0
106608,2064773,1,1,1.0
106609,2064774,1,1,1.0
